# Automate the boring Stuff with python
## Chess Dictionary Validator
{'1h': 'bking', '6c': 'wqueen', '2g': 'bbishop', '5h': 'bqueen', '3e': 'wking'} to represent a chess board.

In [103]:
from typing import Dict
# VALID_SPACE = []
# for i in range(1,9):
#     for j in ['a', 'b', 'c', 'd', 'e','f','g','h']:
#         VALID_SPACE.append(f"{i}{j}")

# Updated chatgpt code
VALID_SPACE = {f"{i}{j}" for i in range(1, 9) for j in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']}

VALID_PIECES = {'king': 1, 'queen': 1, 'knight':2, 'bishop': 2, 'rook': 2, 'pawn':8}

# ------------------------

current_board_position = {'1h': 'bking', '6c': 'wqueen', '2g': 'bbishop', '5h': 'bqueen', '3e': 'wking'}
def isValidChessBoard(current_board_pos: Dict):
    for curr_pos in current_board_pos.keys():
        if curr_pos not in VALID_SPACE:
            raise ValueError("INVALID Position OCCUPIED")
    
    # Since it's a dictionary, and no two keys can be same, we can skip the check of having two pieces on same square.
    white_pieces = {}
    black_pieces = {}
    for piece in current_board_pos.values():
        if piece[0] == 'w':
            white_pieces[piece[1:]] = white_pieces.get(piece[1:], 0)+1
        elif piece[0] == 'b':
            black_pieces[piece[1:]] = black_pieces.get(piece[1:], 0)+1
        else:
            raise ValueError(f"""INVALID COLOR INPUT. Neither Black nor White:" '"{piece[0]}"' value should be only w or b""")
    
    if 'king' not in white_pieces.keys():
        raise ValueError("Missing White King")
    if 'king' not in black_pieces.keys():
        raise ValueError("Missing Black King")
    
    for k,v in white_pieces.items():
        if k not in VALID_PIECES.keys():
            raise ValueError("White Pieces are invalid")
        if white_pieces[k] > VALID_PIECES[k]:
            raise ValueError("Extra White Pieces")

    for k,v in black_pieces.items():
        if k not in VALID_PIECES.keys():
            raise ValueError("Black Pieces are invalid")
        if black_pieces[k] > VALID_PIECES[k]:
            raise ValueError("Extra Black Pieces")
    
    return "True"

isValidChessBoard(current_board_position)


'True'

🔍 1. You used sets for VALID_SPACE — smart. But is this optimal?
You're using 'a' to 'h' manually. Why not generate that? Think of something like chr() or string.ascii_lowercase[:8].<br>
==>> It sure works. <br> Thanks
Could VALID_SPACE be computed on the fly instead of stored? <br>
==> It can for sure, but the loops will take extra time. 

Or... flip it: Why validate positions against a hardcoded list at all? Think dynamic constraints. <br>
==>> Hmm... that's true. I can just check if the keys[0] are between 1 to 8 and keys[1] are between 'a' to 'h'.

🤔 2. You split white/black piece validation manually. Could it be DRY-er? <br>
Notice how white_pieces and black_pieces repeat the same loop logic.<br>
Yes they do repeat!!!<br>

Can you compress that logic? Maybe loop over colors in ('w', 'b')?<br>
==>>

```
pieces = {
    'w': {},
    'b': {}
}
for piece in current_board_pos.values():

    color = piece[0]

    ptype = piece[1:]

    if color not in ('w', 'b'):
        raise ValueError("Invalid piece color")

    if ptype not in VALID_PIECES:
        raise ValueError(f"Invalid piece type: {ptype}")

    pieces[color][ptype] = pieces[color].get(ptype, 0) + 1
```

Could you build a single function that validates either color's pieces?<br>
==>> Can do but not required.

💡 3. Why raise errors at every check?
Errors are fine for debugging, but would a return False or collection of error messages give you more power later?<br>
==>> Hmm... Got it! 

Would a diagnostic report be more useful if you're analyzing thousands of boards in simulation?<br>
==>> Agreed.

📦 4. Magic strings are a code smell
You’re using lots of "king", "pawn", etc. scattered across the code.

How would this code survive if you decided to allow custom chess variants? <br>
==>> Agreed, but how?? :wondering:

Consider isolating rules into a config or constant dictionary to decouple logic from data.<br>
==>> I see...

🧠 5. Can this be a Class?
You're passing a board to a function, but you’re dealing with a data model + behavior.<br>
==>> It sure can be!!

What if ChessBoard was an object, and is_valid() was a method?<br>
==>> hmm hmm.. gotcha

Think extensibility: What if we later want to validate moves, or score positions, or run simulations?<br>
==>> Hmm... what should I do.

🧪 6. How testable is this?
Can you decouple validation into smaller units so that:

```is_valid_position()```
```validate_piece_count()```
```count_pieces()``` <br>
can all be tested independently? <br>
==>> yeah!! can be done.

Think modular test coverage.

🧨 7. Does this scale?
What if you wanted to validate 10 million boards? Do you think this structure is fast enough? Where’s your bottleneck?

Think profiling: what’s your most expensive operation? Is it in lookups? Dictionary builds? Could caching help?.

==>> Hmmmmmm... I wonder

### Chatgpt Optimized Code

In [94]:
from typing import Dict
import string

VALID_SPACE = {f"{row}{col}" for row in range(1, 9) for col in string.ascii_lowercase[:8]}  # '1a', '8h'
VALID_PIECES = {'king': 1, 'queen': 1, 'rook': 2, 'bishop': 2, 'knight': 2, 'pawn': 8}

def isValidChessBoard(board: Dict[str, str]) -> bool:
    piece_counts = {'w': {}, 'b': {}}
    for pos, piece in board.items():

        if pos not in VALID_SPACE:
            raise ValueError(f"Invalid board position: {pos}")
        
        if len(piece) < 2:
            raise ValueError(f"Invalid piece format: {piece}")
        
        color, ptype = piece[0], piece[1:]

        if color not in piece_counts:
            raise ValueError(f"Invalid piece color: {color}")
        
        if ptype not in VALID_PIECES:
            raise ValueError(f"Invalid piece type: {ptype}")
        
        piece_counts[color][ptype] = piece_counts[color].get(ptype, 0) + 1

    for color in ('w', 'b'):
        if piece_counts[color].get('king', 0) != 1:
            raise ValueError(f"{color.upper()} side must have exactly one king")
        
        total_pieces = sum(piece_counts[color].values())
        
        if total_pieces > 16:
            raise ValueError(f"{color.upper()} side has too many total pieces: {total_pieces}")
        
        for ptype, count in piece_counts[color].items():
            if count > VALID_PIECES[ptype]:
                raise ValueError(f"{color.upper()} has too many {ptype}s")
    return True


In [101]:
board = {
    '1a': 'wking', '1b': 'wqueen', '1c': 'wbishop', '1d': 'wbishop',
    '1e': 'wrook', '1f': 'wrook', '1g': 'wknight', '1h': 'wknight',
    '2a': 'wpawn', '2b': 'wpawn', '2c': 'wpawn', '2d': 'wpawn',
    '2e': 'wpawn', '2f': 'wpawn', '2g': 'wpawn', '2h': 'wpawn',
    '8a': 'bking'
}

print(isValidChessBoard(board))  # Should return True

True


## Fantasy Game Inventory

In [109]:
inv = {'rope': 1, 'torch': 6, 'gold coin': 42, 'dagger': 1, 'arrow': 12}
def displayInventory(inv: Dict):
    print("Inventory")
    for k,v in inv.items():
        print(f"- {v} {k}")
    print(f"Total Number of items: {sum(inv.values())}")
displayInventory(inv)

Inventory
- 1 rope
- 6 torch
- 42 gold coin
- 1 dagger
- 12 arrow
Total Number of items: 62


### List to Dictionary Function for Fantasy Game Inventory

In [137]:
dragonLoot = ['gold coin', 'dagger', 'gold coin', 'gold coin', 'ruby']
def addToInventory(inventory: Dict, addedItems):
    copy_inv = inventory.copy()
    for item in addedItems:
        copy_inv[item] = copy_inv.get(item, 0)+1
    return copy_inv

inv = {'gold coin': 42, 'rope': 1}
x = addToInventory(inv, dragonLoot)
displayInventory(x)
print(inv)


Inventory
- 45 gold coin
- 1 rope
- 1 dagger
- 1 ruby
Total Number of items: 48
{'gold coin': 42, 'rope': 1}


In [138]:
def printTable(tableData):
    # Find max width of each column
    colWidths = [max(len(item) for item in col) for col in tableData]
    numRows = len(tableData[0])
    numCols = len(tableData)
    
    for row in range(numRows):
        line = ""
        for col in range(numCols):
            # Right-justify each item in its column width
            line += tableData[col][row].rjust(colWidths[col]) + " "
        print(line.rstrip())

# Example usage:
tableData = [
    ['apples', 'oranges', 'cherries', 'banana'],
    ['Alice', 'Bob', 'Carol', 'David'],
    ['dogs', 'cats', 'moose', 'goose']
]
printTable(tableData)


  apples Alice  dogs
 oranges   Bob  cats
cherries Carol moose
  banana David goose


In [139]:
!pip install zombiedice

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for zombiedice: filename=zombiedice-0.1.6-py3-none-any.whl size=102208 sha256=2db6e972555e2d62b70b28a1e86dbce7bc79f3c64b4efc22090871ee011b3481
  Stored in directory: /Users/vishalkumar/Library/Caches/pip/wheels/e6/e5/43/6f66f133f4205717cce50bbdc13ed05398689ef675fe4be05b
Successfully built zombiedice


In [140]:
import zombiedice
zombiedice.demo()

Zombie Dice Visualization is running. Open your browser to http://localhost:58523 to view it.
Press Ctrl-C to quit.


SystemExit: Quitting...

/opt/anaconda3/envs/mlai_1/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [167]:
import zombiedice

class MyZombie:
    def __init__(self, name):
        # All zombies must have a name:
        self.name = name

    def turn(self, gameState):
        # gameState is a dict with info about the current state of the game.
        # You can choose to ignore it in your code.

        diceRollResults = zombiedice.roll() # first roll
        # roll() returns a dictionary with keys 'brains', 'shotgun', and
        # 'footsteps' with how many rolls of each type there were.
        # The 'rolls' key is a list of (color, icon) tuples with the
        # exact roll result information.
        # Example of a roll() return value:
        # {'brains': 1, 'footsteps': 1, 'shotgun': 1,
        #  'rolls': [('yellow', 'brains'), ('red', 'footsteps'),
        #            ('green', 'shotgun')]}

        # REPLACE THIS ZOMBIE CODE WITH YOUR OWN:
        brains = 0
        while diceRollResults is not None:
            brains += diceRollResults['brains']

            if brains < 3:
                diceRollResults = zombiedice.roll() # roll again
            else:
                break

zombies = (
    zombiedice.examples.RandomCoinFlipZombie(name='Random'),
    zombiedice.examples.RollsUntilInTheLeadZombie(name='Until Leading'),
    zombiedice.examples.MinNumShotgunsThenStopsZombie(name='Stop at 2 Shotguns', minShotguns=2),
    zombiedice.examples.MinNumShotgunsThenStopsZombie(name='Stop at 1 Shotgun', minShotguns=1),
    MyZombie(name='My Zombie Bot'),
    # Add any other zombie players here.
)

# Uncomment one of the following lines to run in CLI or Web GUI mode:
zombiedice.runTournament(zombies=zombies, numGames=1000)
# zombiedice.runWebGui(zombies=zombies, numGames=1000)

Tournament of 1000 games started...
Tournament results:
Wins:
    Stop at 2 Shotguns  349
         My Zombie Bot  236
         Until Leading  199
     Stop at 1 Shotgun  154
                Random   57
Ties:
    Stop at 2 Shotguns    4
     Stop at 1 Shotgun    3
         My Zombie Bot    2
         Until Leading    1
                Random    0
